In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# installing necessary applications for 
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/"
!sudo apt-get update
!sudo apt-get -y install cuda

# Install JoeyNMT
! git clone https://github.com/may-/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.9.0
! pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

--2021-09-21 23:58:06--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2021-09-21 23:58:06 (15.1 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

Executing: /tmp/apt-key-gpghome.6hVX8AEOfN/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub'
gpg: key F60F4B3D7FA2AF80: "cudatools <cudatools@nvidia.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Er

In [3]:
import os
from os import path
tag = "pmi_nhs_da_bt_retrain"
tokenized_data = "/content/drive/MyDrive/DA/updated_data_tok_files"
src_lang = "hi"
tgt_lang = "en"
bpe_operations = 5000
bpe_file = "len_full.bpe.codes." + str(bpe_operations)
bpe_folder = src_lang + "_" + tgt_lang + "_" + tag + "_" + str(bpe_operations)
bpe_data = "/content/drive/MyDrive/DA/" + bpe_folder + "/"

os.environ["src_lang"] = src_lang
os.environ["tgt_lang"] = tgt_lang
os.environ["tokenized_data"] = tokenized_data
os.environ["bpe_file"] = bpe_file
os.environ["bpe_operations"] = str(bpe_operations)
os.environ["jnmt_data_path"] = path.join("joeynmt", "data", src_lang + tgt_lang)

baseline_model_data = "/content/drive/MyDrive/DA/backtranslationhi_en_pmi_nhs_backtranslate_5000/model_data"

os.environ["baseline_model_data"] = baseline_model_data

check_pnt = "/content/drive/MyDrive/DA/model_pmi_nhs_backtranslate_100epochs_new/hien_transformer_pmi_nhs_backtranslate/best.ckpt"

! mkdir -p $jnmt_data_path
! mkdir -p $bpe_data

In [4]:
!mkdir health_domain_data
!cp $tokenized_data/health_concat_filtered_tok.* $tokenized_data/test_tok* /content/health_domain_data

In [5]:
!wc -l /content/health_domain_data/*

   6582 /content/health_domain_data/health_concat_filtered_tok.en
   6582 /content/health_domain_data/health_concat_filtered_tok.hi
    598 /content/health_domain_data/test_tok.en
    598 /content/health_domain_data/test_tok.hi
  14360 total


In [6]:
import pandas as pd

def create_df(src_file, tgt_file):
  src = []
  tgt = []

  with open(src_file, 'r') as src_, open(tgt_file, 'r') as tgt_:
    src_lines = src_.readlines()
    tgt_lines = tgt_.readlines()

  for line_index in range(len(src_lines)):
    src.append(src_lines[line_index].strip('\n'))
    tgt.append(tgt_lines[line_index].strip('\n'))

  df = pd.DataFrame({'src_sentence': src,
                     'tgt_sentence': tgt})

  return df

In [8]:
df_train_health = create_df("/content/health_domain_data/health_concat_filtered_tok.en", "/content/health_domain_data/health_concat_filtered_tok.hi" )

In [9]:
df_train_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6582 entries, 0 to 6581
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   src_sentence  6582 non-null   object
 1   tgt_sentence  6582 non-null   object
dtypes: object(2)
memory usage: 103.0+ KB


In [10]:
### splitting data 500 rows from train data for validation purpose
from sklearn.model_selection import train_test_split

df_train, df_dev = train_test_split(df_train_health, test_size=0.076, random_state=42, shuffle=True)

df_dev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 611 to 5706
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   src_sentence  501 non-null    object
 1   tgt_sentence  501 non-null    object
dtypes: object(2)
memory usage: 11.7+ KB


In [11]:
def create_tok_files(input_df, dataset_name):
  src_tok_file = dataset_name + "_tok.en"
  tgt_tok_file = dataset_name + "_tok.hi"
  src_content = input_df['src_sentence'].to_list()
  tgt_content = input_df['tgt_sentence'].to_list()

  with open(src_tok_file, 'w') as src, open(tgt_tok_file, 'w') as tgt:
    for src_line, tgt_line in zip(src_content, tgt_content):
      src.write(src_line + "\n")
      tgt.write(tgt_line + "\n")


In [12]:
create_tok_files(df_train, "train")
create_tok_files(df_dev, "dev")

In [13]:
!cp /content/health_domain_data/*test* .

In [14]:
### Learn BPEs on training data
from os import path

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang < train_tok.$src_lang > train.bpe.$src_lang
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$tgt_lang < train_tok.$tgt_lang > train.bpe.$tgt_lang

! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang < dev_tok.$src_lang > dev.bpe.$src_lang
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$tgt_lang < dev_tok.$tgt_lang > dev.bpe.$tgt_lang


! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang < test_tok.$src_lang > test.bpe.$src_lang
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$tgt_lang < test_tok.$tgt_lang > test.bpe.$tgt_lang


In [15]:
!cp *.bpe.* $jnmt_data_path

!cp *.bpe.* $bpe_data

!cp $baseline_model_data/vocab.txt $jnmt_data_path

In [16]:
model_type = "bpe"
use_cuda = True

# Create the Joey NMT config file

config = """
name: "{name}_transformer"

data:
    src: "{src_lang}"
    trg: "{tgt_lang}"
    train: "data/{name}/train.{model_type}"
    dev:   "data/{name}/dev.{model_type}"
    test:   "data/{name}/test.{model_type}"
    level: "{model_type}"
    lowercase: False
    max_sent_length: 60
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "{check_pnt}"
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    learning_rate_warmup: 1000 
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 512
    batch_type: "token"
    eval_batch_size: 300
    eval_batch_type: "token"
    eval_metric: "bleu"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 100            # TODO: Set to at least once per epoch.
    logging_freq: 100
    model_dir: "models/{name}_transformer_{tag}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: {use_cuda}
    max_output_length: 130
    print_valid_sents: [0, 1, 2, 3]
    keep_best_ckpts: 3
    
model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name = src_lang+tgt_lang, src_lang = src_lang, tgt_lang = tgt_lang, model_type = model_type, use_cuda = use_cuda, tag = tag, check_pnt = check_pnt)
with open("joeynmt/configs/transformer_{name}.yaml".format(name = src_lang+tgt_lang), 'w') as f:
    f.write(config)

In [17]:
! pip install --upgrade sacrebleu==2.0.0

In [18]:
# Train the model
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src_lang$tgt_lang.yaml

2021-09-22 00:07:21,988 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-22 00:07:22,041 - INFO - joeynmt.data - Loading training data...
2021-09-22 00:07:22,156 - INFO - joeynmt.data - Building vocabulary...
2021-09-22 00:07:22,584 - INFO - joeynmt.data - Loading dev data...
2021-09-22 00:07:22,590 - INFO - joeynmt.data - Loading test data...
2021-09-22 00:07:22,595 - INFO - joeynmt.data - Data loaded.
2021-09-22 00:07:22,596 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-22 00:07:22,793 - INFO - joeynmt.model - Enc-dec model built.
2021-09-22 00:07:24,705 - INFO - joeynmt.training - Total params: 12422400
2021-09-22 00:07:29,538 - INFO - joeynmt.training - Loading model from /content/drive/MyDrive/DA/model_pmi_nhs_backtranslate_100epochs_new/hien_transformer_pmi_nhs_backtranslate/best.ckpt
2021-09-22 00:07:32,474 - INFO - joeynmt.helpers - cfg.name                           : hien_transformer
2021-09-22 00:07:32,474 - INFO - joeynmt.helpers - cfg.

In [20]:
copy_model_to = "/content/drive/MyDrive/DA/model_" + tag + "_30epochs"
!mkdir -p $copy_model_to
os.environ["copy_model_to"] = copy_model_to

!cp -R /content/joeynmt/models/hien_transformer_pmi_nhs_da_bt_retrain/ $copy_model_to

In [21]:
# Output our validation accuracy
! cat "/content/joeynmt/models/hien_transformer_pmi_nhs_da_bt_retrain/validations.txt"

Steps: 73100	Loss: 46507.32422	PPL: 14.37267	bleu: 10.02207	LR: 0.00030000	
Steps: 73200	Loss: 43876.79688	PPL: 12.36134	bleu: 11.34310	LR: 0.00030000	
Steps: 73300	Loss: 42378.65625	PPL: 11.34430	bleu: 11.92583	LR: 0.00030000	
Steps: 73400	Loss: 41173.03516	PPL: 10.58694	bleu: 11.77921	LR: 0.00030000	
Steps: 73500	Loss: 40478.01562	PPL: 10.17354	bleu: 13.17282	LR: 0.00030000	
Steps: 73600	Loss: 39628.21094	PPL: 9.68993	bleu: 12.85233	LR: 0.00021000	
Steps: 73700	Loss: 39269.63672	PPL: 9.49284	bleu: 13.19652	LR: 0.00021000	
Steps: 73800	Loss: 38989.87891	PPL: 9.34186	bleu: 13.54766	LR: 0.00021000	
Steps: 73900	Loss: 38749.35938	PPL: 9.21397	bleu: 13.24761	LR: 0.00021000	
Steps: 74000	Loss: 38249.79297	PPL: 8.95391	bleu: 13.87664	LR: 0.00021000	
Steps: 74100	Loss: 38097.27734	PPL: 8.87599	bleu: 13.74673	LR: 0.00021000	
Steps: 74200	Loss: 37557.62109	PPL: 8.60568	bleu: 14.29028	LR: 0.00014700	
Steps: 74300	Loss: 37796.37500	PPL: 8.72424	bleu: 14.47495	LR: 0.00014700	
Steps: 74400	Loss: 3

In [24]:
# Test our model
! cd joeynmt; python3 -m joeynmt test /content/joeynmt/configs/transformer_hien.yaml --output_path hi_en_da2

2021-09-22 01:59:18,148 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-22 01:59:18,149 - INFO - joeynmt.data - Building vocabulary...
2021-09-22 01:59:18,570 - INFO - joeynmt.data - Loading dev data...
2021-09-22 01:59:18,577 - INFO - joeynmt.data - Loading test data...
2021-09-22 01:59:18,583 - INFO - joeynmt.data - Data loaded.
2021-09-22 01:59:18,598 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 300
2021-09-22 01:59:18,598 - INFO - joeynmt.prediction - Loading model from models/hien_transformer_pmi_nhs_da_bt_retrain/90400.ckpt
2021-09-22 01:59:21,493 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-22 01:59:21,687 - INFO - joeynmt.model - Enc-dec model built.
2021-09-22 01:59:21,753 - INFO - joeynmt.prediction - Decoding on dev set (data/hien/dev.bpe.en)...
2021-09-22 02:00:06,497 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-09-22 02:00:06,497 - WARNING - sacrebleu - It 

In [23]:
!python3 /content/joeynmt/scripts/plot_validations.py /content/joeynmt/models/hien_transformer_pmi_nhs_da_bt_retrain --plot_values bleu PPL --output_path bleu-ppl.png